In [29]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
import requests
import pymongo


In [37]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [38]:
# URL of page to be scraped
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)
nasa_html = browser.html
# Create BeautifulSoup object; parse with 'html'
soup = BeautifulSoup(nasa_html,'html.parser')

In [39]:

# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text.
u_list = soup.find("ul", class_= "item_list")
li_list = u_list.find("li", class_="slide" )
title_results = li_list.find('div', class_='content_title').text
article_teaser = li_list.find('div', class_="article_teaser_body").text
print(title_results)
print(article_teaser)

Mars Helicopter Attached to NASA's Perseverance Rover
The team also fueled the rover's sky crane to get ready for this summer's history-making launch.


In [40]:
# Visit the url for JPL Featured Space Image and produce the daily Featured Image
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(featured_image_url)
browser.click_link_by_partial_text('FULL IMAGE')

jplimage_html = browser.html
jplimage_soup = BeautifulSoup(jplimage_html,'html.parser')

img = jplimage_soup.find('img', class_='thumb')['src']
image_path = f'https://www.jpl.nasa.gov{img}'
print(image_path)


https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23835-640x350.jpg


In [41]:
featured_image_url =image_path

In [21]:
# Search through @MarsWxReport twitter page to produce Mar's daily weather reports tweets in text
mars_twitter_url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(mars_twitter_url)

mars_twitter_soup = BeautifulSoup(response.text,'lxml')

mars_tweet = mars_twitter_soup.find_all('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")
current_weather_tweet = mars_tweet[0].text
current_weather_tweet


'InSight sol 485 (2020-04-07) low -93.4ºC (-136.1ºF) high -7.0ºC (19.4ºF)\nwinds from the WNW at 5.2 m/s (11.5 mph) gusting to 17.4 m/s (38.9 mph)\npressure at 6.50 hPapic.twitter.com/lSCXda8hgu'

In [22]:
mars_weather = current_weather_tweet

In [23]:
# read_html function in Pandas to automatically scrape any tabular data from a page.
mars_facts = "https://space-facts.com/mars/"
fact_tables = pd.read_html(mars_facts)
fact_tables
mars_table = fact_tables[0]

# to_html method that we can use to generate HTML tables from DataFrames.
mars_facts_html = mars_table.to_html()
# strip unwanted newlines to clean up the table.
mars_facts_html.replace('\n', '')
mars_facts_html


'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    <

In [24]:
mars_space_facts = mars_facts_html

In [25]:
# Search through astrogeology.usgs.gov for class to the 4 enhanced images of Mars Hemispere links
mars_hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(mars_hemisphere_url)
mars_html = browser.html
html_soup = BeautifulSoup(mars_html, "html.parser")
hemis = html_soup.find_all("div", class_="item")

# create dict
hemisphere_image_urls = []
#  for loop to comb the page and get the Text/Name of the image hemisphere, the download link and
#     the create the full url of the image.
for hemi in hemis:
    image_title = hemi.find('h3').text
    url_link = hemi.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + url_link 
# visits the full url of the image and returns the image,title and url 
    browser.visit(image_link)
    html= browser.html
    link_soup = BeautifulSoup(html, "html.parser")
    image_download = link_soup.find('div',class_= 'downloads')
    image_url = image_download.find('a')['href']
    hemisphere_image_urls.append({"Title:":image_title ,"Image URL:":image_url})
hemisphere_image_urls

[{'Title:': 'Cerberus Hemisphere Enhanced',
  'Image URL:': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'Title:': 'Schiaparelli Hemisphere Enhanced',
  'Image URL:': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'Title:': 'Syrtis Major Hemisphere Enhanced',
  'Image URL:': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'Title:': 'Valles Marineris Hemisphere Enhanced',
  'Image URL:': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]